In [1]:
import numpy as np

joint_prob_table = np.array([(0.3,0.3,0), (0.1,0.2,0.1)], dtype = float)
#Calculate marginals
py = np.array(np.sum(joint_prob_table, axis = 0))
px = np.array(np.sum(joint_prob_table, axis = 1))
#Domains
domx = np.array([1,2]).reshape(1,2)
domy = np.array([-1,0,5]).reshape(1,3)
#Calculate conditionals
y_x = np.divide(joint_prob_table,px.reshape(2,1))
x_y = np.divide(joint_prob_table,py.reshape(1,3))

print("MARGINALS")
print("X:       ",px)
print("Y:       ",py)
print("CONDITIONALS")
print("X|Y:     \n",x_y)
print("Y|X:     \n",y_x)

MARGINALS
X:        [ 0.6  0.4]
Y:        [ 0.4  0.5  0.1]
CONDITIONALS
X|Y:     
 [[ 0.75  0.6   0.  ]
 [ 0.25  0.4   1.  ]]
Y|X:     
 [[ 0.5   0.5   0.  ]
 [ 0.25  0.5   0.25]]


In [32]:
def expectations(fx,px):
    res = np.dot(fx,px.T)
    return res
def cond_exp(domx,x_y,key):
    if key == 'x':
        return np.sum(domx.T * x_y, axis=0)
    else:
        return np.sum(domx * x_y, axis=1)
def covariance(x,y,mx,my,xy):
    mx_x = x - mx
    my_y = y - my
    tmp = mx_x.T * my_y
    return np.sum(tmp * xy)

In [33]:
ex_x = expectations(domx,px)
print("EXPECTATIONS")
print("X:    ",ex_x)
ex_y = expectations(domy,py)
print("Y:    ",ex_y)
ex_x_y = cond_exp(domx,x_y,'x')
print("X|Y:    \n",ex_x_y)
ex_y_x = cond_exp(domy,y_x,'y')
print("Y|X:    \n",ex_y_x)
print("COVARIANCE")
cov = covariance(domx,domy,ex_x,ex_y,joint_prob_table)
print(cov)

EXPECTATIONS
X:     [ 1.4]
Y:     [ 0.1]
X|Y:    
 [ 1.25  1.4   2.  ]
Y|X:    
 [-0.5  1. ]
COVARIANCE
0.36


In [34]:
def entropy(p):
    return -np.sum(p * np.log(p))
def joint_entropy(p):
    _p = p.reshape(p.shape[0] * p.shape[1])
    log2p = np.zeros(shape=_p.shape,dtype = np.float32)
    for i in range(len(_p)):
        if _p[i] == 0:
            log2p[i] = 0
        else:
            log2p[i] = np.log(_p[i])
    log2p = log2p.reshape(p.shape[0],p.shape[1])
    return -np.sum(p * log2p)
def cond_entropy(jp,cp):
    
    _cp = cp.reshape(cp.shape[0] * cp.shape[1])
    log2cp = np.zeros(shape=_cp.shape,dtype = np.float32)
    for i in range(len(_cp)):
        if _cp[i] == 0:
            log2cp[i] = 0
        else:
            log2cp[i] = np.log(_cp[i])
    log2cp = log2cp.reshape(cp.shape[0],cp.shape[1])
    return -np.sum(jp * log2cp)

In [35]:
hxy = joint_entropy(joint_prob_table)
print ("Joint Entropy:    ",hxy)
print("MARGINAL ENTROPIES")
hx = entropy(px)
print("H(x):    ",hx)
hy = entropy(py)
print("H(y):    ",hy)

Joint Entropy:     1.50478830338
MARGINAL ENTROPIES
H(x):     0.673011667009
H(y):     0.943348392329


In [36]:
print("CONDITIONAL ENTROPIES")
hy_x = cond_entropy(joint_prob_table,y_x)
print ("H(Y|X):    ",hy_x)
hx_y = cond_entropy(joint_prob_table,x_y)
print ("H(X|Y):    ",hx_y)

CONDITIONAL ENTROPIES
H(Y|X):     0.831776618958
H(X|Y):     0.561439904571


In [40]:
def mutual_info(x,y,jxy):
    xy = x.reshape(x.shape[0],1) * y.reshape(1,y.shape[0])
    logxy = np.log(xy)
    logjxy = np.log(jxy)
    tmp = logjxy - logxy
    res = 0
    jxy = jxy.reshape(jxy.shape[0] * jxy.shape[1])
    tmp = tmp.reshape(tmp.shape[0] * tmp.shape[1])
    for i in range(len(tmp)):
        if jxy[i] == 0:
            continue
        else:
            res += jxy[i] * tmp[i]
    jxy = jxy.reshape(x.shape[0],y.shape[0])
    return res

In [41]:
I = mutual_info(px,py,joint_prob_table)
print ("I(x,y):    ",I)

I(x,y):     0.111571775657


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [42]:
print(hx + hy - hxy)

0.111571755963
